##### --- Day 15: Warehouse Woes ---

--- Part One ---

In [1]:
import itertools

file = open("day_15_r.txt", "r")
instructions = "".join([i.strip() for i in file.readlines()])

file = open("day_15_m.txt", "r")
warehouse = [[i for i in j.strip()] for j in file.readlines()]

In [2]:
def next_pos(pos, d):
    # Returns next position, given current position and a direction.
    if d == "^":
        return (pos[0]-1, pos[1])
    if d == "v":
        return (pos[0]+1, pos[1])
    if d == "<":
        return (pos[0], pos[1]-1)
    if d == ">":
        return (pos[0], pos[1]+1)

def is_pushable(pos, direction, the_map, crates_pushed):
    # Recursively check if the crate is pushable.
    next_y, next_x = next_pos(pos, d)
    if warehouse[next_y][next_x] == "#":
        # Not pushable
        return 0
    elif warehouse[next_y][next_x] == ".":
        return crates_pushed
    else:
        # Add one to crates pushed and keep pushing.
        crates_pushed += 1
        new_pos = next_y, next_x
        return is_pushable(new_pos, direction, the_map, crates_pushed)

def calculate_gps(a_map):
    gps = 0
    for y_ind, r in enumerate(a_map):
        for x_ind, l in enumerate(r):
            if l == "O" or l == "[":
                gps += 100 * y_ind + x_ind
    print(gps)

# Find current position of the robot.
for y_ind, r in enumerate(warehouse):
    for x_ind, l in enumerate(r):
        if l == "@":
            current_pos = [y_ind, x_ind]
            
for d in instructions:
    next_y, next_x = next_pos(current_pos, d)
    if warehouse[next_y][next_x] == ".":
        # Space is free, move to it.
        warehouse[next_y][next_x] = "@"
        warehouse[current_pos[0]][current_pos[1]] = "."
        current_pos = (next_y, next_x)
    elif warehouse[next_y][next_x] == "#":
        # Imovable object, no movement.
        pass
    else:
        # Crate encountered, check if pushable and push if needed.
        if pushes := is_pushable(current_pos, d, warehouse, 0):
            warehouse[next_y][next_x] = "@"
            warehouse[current_pos[0]][current_pos[1]] = "."
            current_pos = (next_y, next_x)
            to_be_crate = current_pos
            for i in range(pushes):
                to_be_crate = next_pos(to_be_crate, d)
            warehouse[to_be_crate[0]][to_be_crate[1]] = "O"

calculate_gps(warehouse)   

1412971


--- Part Two ---

In [12]:
file = open("day_15_m.txt", "r")
warehouse = [_ for _ in file.readlines()]

file = open("day_15_r.txt", "r")
instructions = "".join([i.strip() for i in file.readlines()])

# Double widths and convert to grid format.
warehouse = [i.replace("#", "##").replace("O", "[]").replace(".", "..").replace("@", "@.") for i in warehouse]
warehouse = [[i for i in j.strip()] for j in warehouse]

In [13]:
current_pos = [24, 48]
for d in instructions:
    next_y, next_x = next_pos(current_pos, d)
    if warehouse[next_y][next_x] == ".":
        # Space is free, move to it.
        warehouse[next_y][next_x] = "@"
        warehouse[current_pos[0]][current_pos[1]] = "."
        current_pos = (next_y, next_x)
    elif warehouse[next_y][next_x] == "#":
        # Imovable object, no movement.
        pass
    else:
        # Crate encountered, check if pushable and push if needed.
        if d in "<>":
            if pushes := is_pushable(current_pos, d, warehouse, 0):
                # Move the crates left or right.
                if d == "<":
                    warehouse[current_pos[0]][current_pos[1] + (-1*pushes) - 1 : current_pos[1] -1 ] = warehouse[current_pos[0]][current_pos[1] + (-1*pushes): current_pos[1]]
                else:
                    warehouse[current_pos[0]][current_pos[1]+2: current_pos[1] + pushes+2] = warehouse[current_pos[0]][current_pos[1]+1: current_pos[1] + pushes+1]
                    
                warehouse[next_y][next_x] = "@"
                warehouse[current_pos[0]][current_pos[1]] = "."
                current_pos = (next_y, next_x)
        else:
            if d ==  "^":
                y_chng = -1
            else:
                y_chng = 1

            # Find crates to move.
            layers = vertical_crates(current_pos, d, warehouse)
            
            if layers:
                # Move crates up or down.
                for layer in layers[::-1]:
                    for c in layer:
                        warehouse[c[0][0]+y_chng][c[0][1]] = "["
                        warehouse[c[1][0]+y_chng][c[1][1]] = "]"
                        warehouse[c[0][0]][c[0][1]] = "."
                        warehouse[c[1][0]][c[1][1]] = "."

                warehouse[next_y][next_x] = "@"
                warehouse[current_pos[0]][current_pos[1]] = "."
                current_pos = (next_y, next_x)

calculate_gps(warehouse)

1429299


In [14]:
def vertical_crates(current_pos, d, warehouse):
    # Check if the crate is pushable.
    layers = [] # In format [ [ [[l1,c1],[l1,c1]], [l1,c2],[l1,c2] ], ... ]

    # Put first crate in layers.
    layers.append([crate_there(current_pos, d, warehouse)])
    level = 0
    while True:
        new_layer = []
        for crate in layers[level]:
            for c in crate:
                n_y, n_x = next_pos(c, d)
                if warehouse[n_y][n_x] == "#":
                    # Not pushable, return 0 crates to push.
                    return 0
                if crate_there((c[0],c[1]), d, warehouse):
                    new_layer.append(crate_there((c[0],c[1]), d, warehouse))
        if len(new_layer) > 0:
            # Add layer of new crates and increase level after stripping out duplicate crates.
            new_layer.sort()
            layers.append(list(new_layer for new_layer,_ in itertools.groupby(new_layer)))
            # Go up one level.
            level += 1
        else:
            # No more crates, so return all of the crates scheduled to be pushed.
            return layers
        
                            
def crate_there(pos, d, warehouse):
    # Return the two coordinates of a crate if one is above or below.
    y, x = next_pos(pos, d)
    if warehouse[y][x] == "[":
        return ([y, x], [y, x+1])
    if warehouse[y][x] == "]":  
        return ([y, x-1], [y, x])
    return False